# Execution time

We first compare **performance** (execution time) of the gemm implementation in CLBlast with row-major and column-major matrices as input.

## Includes

### Standard

In [ ]:
import os
import sys
import json

### Scientific

In [ ]:
import IPython as ip
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mp

In [ ]:
print('IPython version: %s' % ip.__version__)
print('NumPy version: %s' % np.__version__)
print('SciPy version: %s' % sp.__version__)
print('Pandas version: %s' % pd.__version__)
print('Matplotlib version: %s' % mp.__version__)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
# import scipy.stats as st

### Collective Knowledge

In [ ]:
import ck.kernel as ck
print('CK version: %s' % ck.__version__)

## Access experimental results

In [ ]:
def get_experimental_results(tags):
    r=ck.access({'action':'search', 'module_uoa':'experiment', 'tags':tags})
    if r['return']>0:
        print ("Error: %s" % r['error'])
        exit(1)
    # FIXME: For now, assume a single entry per the given tags.
    r=ck.access({'action':'list_points', 'module_uoa':'experiment', 'data_uoa': r['lst'][0]['data_uoa']})
    if r['return']>0:
        print ("Error: %s" % r['error'])
        exit(1)
    results = {}
    for point in r['points']:
        with open(os.path.join(r['path'], 'ckp-%s.0001.json' % point)) as point_file:
            point_data_raw = json.load(point_file)
            run_info_list = [
                    characteristics['run']
                    for characteristics in point_data_raw['characteristics_list']
                    if characteristics['run']['run_success'] == 'yes'
                ]
            point_data_dict = {
                    'ms'   : [ run_info.get('ms_1',0)      for run_info in run_info_list ],
#                     'layout' : [ run_info.get('layout',0)    for run_info in run_info_list ],
                }
            layout_info = point_data_raw['choices']['env']['CLBLAST_LAYOUT']
            if layout_info == 101:
                results["row-major"] = point_data_dict
            elif layout_info == 102:
                results["col-major"] = point_data_dict
    return results

In [ ]:
results = get_experimental_results('layout-options,clblast')
results

In [ ]:
# compute median
flattenToFloat = lambda x : float(x[0])
d = { k : np.median(list(map(flattenToFloat, v['ms']))) for (k,v) in results.items()}
d

In [ ]:
df = pd.DataFrame.from_dict(d, orient='index')
df.index.name = 'variations'
df.columns.name = 'runtime (ms)'
df

In [ ]:
df.T \
    .plot(title='Execution time (ms)',
          kind='bar', rot=0, ylim=[0,45], figsize=[12, 6], grid=True, legend=True, colormap=cm.autumn)